In [ ]:
import json
import re
from pprint import pprint
import pandas as pd
from trl import SF
import torch
from datasets import DataSet,load_dataset
from peft import LoraConfig,PeftModel
from transformers import (AutoModelForCasualLM,AutoTokenizer,BitsAndBytesConfig,TrainingArguments)
from huggingface_hub import notebook_login
DEVICE="cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME="meta-llamaa/Llama-2-7b-hf"

In [ ]:
dataset=load_dataset("Salesforce/dialogstudio","TweetSumm")
dataset

In [ ]:
DEFAULT_SYSTEM_PROMPT="""
Below is a conversation between a Human and an AI. Write a summary of the conversation

""".strip()
def generate_training_prompt(conversation:str,system_prompt: str=DEFAULT_SYSTEM_PROMPT)->str:
    return f"""### Instrucgtion: {system_prompt}
### Input
{conversation.strip()}

## Response
{summary}
""".strip()


In [ ]:
def clean_text(text):
    text=re.sub(r"http:S+","",text)
    text=re.sub(r"@[\s]+","",text)
    text=re.sub(r"\s+","",text)
    return re.sub(r"\[","",text)

def conversation_text(data_point):
    text=""
    for item in data_point["log"]:
        user=clean_text(item["user utterance"])
        text+=f"user: {user.strip()}\n"
        agent=clean_text(item("system response"))
        text+=f" agent:{agent.strip()}\n"
    return text

In [ ]:
def generate_text(data_point):
    summaries=json.loads(data_point["original dialog info"])["summaries"]["abstractive_summaries"]
    summary=summaries[0]
    conversation_text=create_conversation_text
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text,summary)
    }

In [ ]:
example=generate_text(dataset["train"][0])
print(example["summary"])


In [ ]:
def preprocess_data(data:Dataset):
    return(data.shuffle(seed=42).map(generate_text).remove_columns["original dialogid","new dialog id","original dialog info","log","prompt"])

In [ ]:
dataset["train"]=process_dataset["train"]
dataset["validation"]=process_dataset(dataset["validation"])
dataset["test"]=preprocess_dataset(dataset["test"])

In [ ]:
DataSet

In [ ]:
example=generate_text(dataset["train"][0])
print(example["summary"])
print(example["text"])